# 벡터스토어 기반 검색기(VectorStore-backed Retriever)

**VectorStore 지원 검색기** 는 vector store를 사용하여 문서를 검색하는 retriever입니다.

Vector store에 구현된 **유사도 검색(similarity search)** 이나 **MMR** 과 같은 검색 메서드를 사용하여 vector store 내의 텍스트를 쿼리합니다.


아래의 코드를 실행하여 VectorStore 를 생성합니다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader("./data/2025_사우회선출.txt")

# 문서를 로드합니다.
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# 분할된 텍스트와 임베딩을 사용하여 Chroma 벡터 데이터베이스를 생성합니다.
vector_store = Chroma.from_documents(split_docs, embeddings)

Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 201, which is longer than the specified 100
Created a chunk of size 220, which is longer than the specified 100


In [3]:
split_docs

[Document(metadata={'source': './data/2025_사우회선출.txt'}, page_content='안녕하세요, 2024년 사우회입니다.\n어느덧 2024년이 마무리되어 가는 이 시점에 2025년 사우회를 이끌어주실 분에 대한 투표를 진행하겠습니다.\n모두 한 분 한 분의 소중한 한 표를 꼭 행사해 주시기 바랍니다!'),
 Document(metadata={'source': './data/2025_사우회선출.txt'}, page_content='투표 방식: 무기명 온라인 1인 1투표\n투표 기간: 2024년 12월 23일(월)까지 (1주일간 진행)\n투표 링크: https://forms.gle/ooDQvUP4SzJRaUoS6'),
 Document(metadata={'source': './data/2025_사우회선출.txt'}, page_content='[노사 협의회 구성원 및 선출 공고 사항]\n노사 협의회 구성원 중 근로자 대표는 노사 협의회장을 겸하며, 노사 간 중요 의사결정 과정에서 합의의 주체가 될 수 있습니다.\n다만, 제반 제도와 관련된 안내 및 직원 커뮤니케이션 등의 업무는 여전히 인사팀에서 진행할 예정이오니, 근로자 대표로 선출된 직원분께서는 큰 부담 없이 선출 결과를 받아주시면 감사하겠습니다.'),
 Document(metadata={'source': './data/2025_사우회선출.txt'}, page_content='2025년 근로자(대표) 위원\n사우회장 1명, 총무 1명\n사용자 위원과 근로자 위원이 원활한 의사소통을 통해 상호 간의 이해와 협조를 증진함으로써 노사 공동의 이익을 구현합니다.'),
 Document(metadata={'source': './data/2025_사우회선출.txt'}, page_content='근로자 대표 (회사발전위원회)\n근로기준법상 각종 제도에 대해 사측과 합의를 진행합니다.\n기존 사우회 및 회사발전위원회 위원장 선출 방식과 동일하게 근로자 투표 방식으로 진행됩니다.\n단, 

### VectorStore에서 VectorStoreRetriever 초기화(as_retriever)

`as_retriever` 메서드는 VectorStore 객체를 기반으로 VectorStoreRetriever를 초기화하고 반환합니다. 이 메서드를 통해 다양한 검색 옵션을 설정하여 사용자의 요구에 맞는 문서 검색을 수행할 수 있습니다.

**매개변수(parameters)**

- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
  - `search_kwargs`: 추가 검색 옵션
    - `k`: 반환할 문서 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
    - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링

**반환값(return)**

- `VectorStoreRetriever`: 초기화된 VectorStoreRetriever 객체

**참고**

- 다양한 검색 전략 구현 가능 (유사도, MMR, 임계값 기반)
- MMR (Maximal Marginal Relevance) 알고리즘으로 검색 결과의 다양성 조절 가능
- 메타데이터 필터링으로 특정 조건의 문서만 검색 가능
- `tags` 매개변수를 통해 검색기에 태그 추가 가능

**주의사항**

- `search_type`과 `search_kwargs`의 적절한 조합 필요
- MMR 사용 시 `fetch_k`와 `k` 값의 균형 조절 필요
- `score_threshold` 설정 시 너무 높은 값은 검색 결과가 없을 수 있음
- 필터 사용 시 데이터셋의 메타데이터 구조 정확히 파악 필요
- `lambda_mult` 값이 0에 가까울수록 다양성이 높아지고, 1에 가까울수록 유사성이 높아짐

In [4]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당
retriever = vector_store.as_retriever()

### Retriever의 invoke()

`invoke` 메서드는 Retriever의 주요 진입점으로, 관련 문서를 검색하는 데 사용됩니다. 이 메서드는 동기적으로 Retriever를 호출하여 주어진 쿼리에 대한 관련 문서를 반환합니다.

**매개변수(parameters)**

- `input`: 검색 쿼리 문자열
- `config`: Retriever 구성 (Optional[RunnableConfig])
- `**kwargs`: Retriever에 전달할 추가 인자

**반환값(return)**

- `List[Document]`: 관련 문서 목록

In [5]:
# 관련 문서를 검색
docs = retriever.invoke("사우회장 선출 기준")

for doc in docs:
    print(doc.page_content)
    print("=========================================================")

사우회장 후보
선출 기준: 역임자를 제외한 스펙트라 근속 5~9년 차 사우를 대상으로 선출.
주요 업무: 연중행사 및 사우 경조사(인사팀과 협업) 담당.
근로자 대표 후보 (회사발전위원회)
선출 기준: 역임자를 제외한 스펙트라 근속 10~15년 차 사우를 대상으로 선출.
총무 후보
선출 기준: 역임자를 제외한 스펙트라 근속 1~4년 차 사우를 대상으로 선출.
주요 업무:
사우 경조사비 송금, 생일 축하금, 입사 축하금, 동호회비 송금.
지출 대장 작성 및 회장 업무 지원.


### Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 검색된 문서의 **중복** 을 피하는 방법 중 하나입니다. 

단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 쿼리에 대한 **문서의 관련성** 과 이미 선택된 **문서들과의 차별성을 동시에 고려** 합니다.


- `search_type` 매개변수를 `"mmr"` 로 설정하여 **MMR(Maximal Marginal Relevance)** 검색 알고리즘을 사용합니다.
- `k`: 반환할 문서 수 (기본값: 4)
- `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)

In [6]:
# MMR(Maximal Marginal Relevance) 검색 유형을 지정
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "fetch_k": 9, "lambda_mult": 0.6}
)

# 관련 문서를 검색합니다.
docs = retriever.invoke("사우회장 선출 기준")

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

사우회장 후보
선출 기준: 역임자를 제외한 스펙트라 근속 5~9년 차 사우를 대상으로 선출.
주요 업무: 연중행사 및 사우 경조사(인사팀과 협업) 담당.
투표 방식: 무기명 온라인 1인 1투표
투표 기간: 2024년 12월 23일(월)까지 (1주일간 진행)
투표 링크: https://forms.gle/ooDQvUP4SzJRaUoS6


### 유사도 점수 임계값 검색(similarity_score_threshold)

유사도 점수 임계값을 설정하고 해당 임계값 이상의 점수를 가진 문서만 반환하는 검색 방법을 설정할 수 있습니다.

임계값을 적절히 설정함으로써 **관련성이 낮은 문서를 필터링** 하고, 질의와 **가장 유사한 문서만 선별** 할 수 있습니다.

- `search_type` 매개변수를 `"similarity_score_threshold"` 로 설정하여 유사도 점수 임계값을 기준으로 검색을 수행합니다.

- `search_kwargs` 매개변수에 `{"score_threshold": 0.8}`를 전달하여 유사도 점수 임계값을 0.8로 설정합니다. 이는 검색 결과의 **유사도 점수가 0.8 이상인 문서만 반환됨** 을 의미합니다.


In [ ]:
retriever = vector_store.as_retriever(
    # 검색 유형을 "similarity_score_threshold 으로 설정
    search_type="similarity_score_threshold",
    # 임계값 설정
    search_kwargs={"score_threshold": 0.8},
    # 여기서는 k가 의미가 없어진다. 0.8 이상은 모두 결과로 나온다.
)

# 관련 문서를 검색
for doc in retriever.invoke("사우회장 선출 기준"):
    print(doc.page_content)
    print("=========================================================")

### top_k 설정

검색 시 사용할 `k` 와 같은 검색 키워드 인자(kwargs)를 지정할 수 있습니다.

`k` 매개변수는 검색 결과에서 반환할 상위 결과의 개수를 나타냅니다.

- `search_kwargs`에서 `k` 매개변수를 1로 설정하여 검색 결과로 반환할 문서의 수를 지정합니다.


In [ ]:
# k 설정
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# 관련 문서를 검색
docs = retriever.invoke("사우회장 선출 기준")

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")